In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from pyumi.epw import Epw
import time

### Adding WandB Artifacts

In [1]:
import sys
sys.path.insert(0, "../../ml-for-bem/")
import os
import json
from storage import upload_to_bucket, creds
import wandb

def config_gcs_adc():
    from storage import creds

    # Copies credentials loaded in from env/json, dump them into local storage file
    with open("credentials.json", "w") as f:
        creds = creds.copy()
        for key, val in creds.items():
            if isinstance(val, bytes):
                val = val.decode("utf-8")

            creds[key] = val
        f.write(json.dumps(creds))

    # Set the credentials env variable
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"

config_gcs_adc()


with wandb.init(
    job_type="upload",
    project="ml-for-bem",
    tags=["weather"],
    group="data",
    notes="Adds EPW files for approx 800 cities around the world, along with metadata.",
    name="add-epw-data-to-registry",
) as run:
    artifact = wandb.Artifact(
        name="global-epw-data",
        type="dataset",
        description="Contains a .zip file, global_epws_indexed.zip, with approx. 850 EPWs from around the world. "
        "The data was generated by taking the top 8k cities by population in the world, "
        "checking their climate zones 50 cities by population in each climate zone "
        "(or all of them if there were fewer than 50 from a CZ within the top 8k cities of the world. "
        "  There are additional metadata files which indicate the climate zones, etc.",
    )
    upload_to_bucket(blob_name="weather/global_epws_indexed.csv", file_name="../../ml-for-bem/data/global_epws_indexed.csv")
    upload_to_bucket(blob_name="weather/global_epws_indexed.zip", file_name="../../ml-for-bem/data/epws/global_epws_indexed.zip")
    upload_to_bucket(blob_name="weather/8k_Cities_w_CZ.csv", file_name="../../ml-for-bem/data/8k_Cities_w_CZ.csv")
    upload_to_bucket(blob_name="weather/worldcities.csv", file_name="../../ml-for-bem/data/worldcities.csv")
    upload_to_bucket(blob_name="weather/global_climate_array.npy", file_name="../../ml-for-bem/data/epws/global_climate_array.npy")
    artifact.add_reference("gs://ml-for-bem-data/weather")
    run.log_artifact(artifact)

INFO:Storage:Successfully fetched bucket location
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: szvsw (ml-for-building-energy-modeling). Use `wandb login --relogin` to force relogin


INFO:Storage:Uploading ../../ml-for-bem/data/global_epws_indexed.csv to bucket:weather/global_epws_indexed.csv...
INFO:Storage:Done uploading.
INFO:Storage:Uploading ../../ml-for-bem/data/epws/global_epws_indexed.zip to bucket:weather/global_epws_indexed.zip...
INFO:Storage:Done uploading.
INFO:Storage:Uploading ../../ml-for-bem/data/8k_Cities_w_CZ.csv to bucket:weather/8k_Cities_w_CZ.csv...
INFO:Storage:Done uploading.
INFO:Storage:Uploading ../../ml-for-bem/data/worldcities.csv to bucket:weather/worldcities.csv...
INFO:Storage:Done uploading.
INFO:Storage:Uploading ../../ml-for-bem/data/epws/global_climate_array.npy to bucket:weather/global_climate_array.npy...
INFO:Storage:Done uploading.
wandb: Generating checksum for up to 10000 objects with prefix "weather"... Done. 0.0s


### Fetch CZ ids for Cities

In [ ]:
import requests
from tqdm.notebook import tqdm
import time

# Get the list of all cities
df = pd.read_csv("../../ml-for-bem/data/worldcities.csv").sort_values(by="population",ascending=False)

# Pick The 8000 most populous
df = df[:8000]

# Add a column for CZ
df["CZ"] = ""

# Get the CZ from ubem.io
for ix,row in tqdm(df.iterrows()):
    print(f"Checking CZ for ", row["city"])
    lat,lng = row["lat"], row["lng"]
    try:
        res = requests.get(f"https://app.ubem.io/api/umibuilder/get_climate_zone?Lat={lat}&Lon={lng}")
        df.loc[ix, "CZ"] = list(res.json().values())[0]["CZ"]
    except:
        print("ERROR", row["city"])
        pass
    # Sleep to prevent accidental DDoS
    time.sleep(0.1)

# Save DF
df.to_csv("8k_Cities_w_CZ.csv")

0it [00:00, ?it/s]

Checking CZ for  Tokyo
Checking CZ for  Jakarta
Checking CZ for  Delhi
Checking CZ for  Guangzhou
Checking CZ for  Mumbai
Checking CZ for  Manila
Checking CZ for  Shanghai
Checking CZ for  São Paulo
Checking CZ for  Seoul
Checking CZ for  Mexico City
Checking CZ for  Cairo
Checking CZ for  New York
Checking CZ for  Dhaka
Checking CZ for  Beijing
Checking CZ for  Kolkāta
Checking CZ for  Bangkok
Checking CZ for  Shenzhen
Checking CZ for  Moscow
Checking CZ for  Buenos Aires
Checking CZ for  Lagos
Checking CZ for  Istanbul
Checking CZ for  Karachi
Checking CZ for  Bangalore
Checking CZ for  Ho Chi Minh City
Checking CZ for  Ōsaka
Checking CZ for  Chengdu
Checking CZ for  Tehran
Checking CZ for  Kinshasa
Checking CZ for  Rio de Janeiro
Checking CZ for  Chennai
Checking CZ for  Xi’an
Checking CZ for  Lahore
Checking CZ for  Chongqing
Checking CZ for  Los Angeles
Checking CZ for  Baoding
Checking CZ for  London
Checking CZ for  Paris
ERROR Paris
Checking CZ for  Linyi
Checking CZ for  Dongg

## Get EPWs for Cities

In [5]:
import random
import pandas as pd
from pathlib import Path
from pyumi.epw import Epw
import time

# Get list of 8k most populous cities with their CZs
df = pd.read_csv("8k_Cities_w_CZ.csv")

# Drop any which errored when fetching cz
df = df[~(df["CZ"].isna())]

# Sort by CZ, then population
df = df.sort_values(by=["CZ","population"],ascending=False)


# Make a dict of columns and add a few then create new df
cols = {col: [] for col in df.columns}
cols["idx"] = []
cols["slug"] = []
top_cities = pd.DataFrame(cols)

# Get the nth most populous city in each climate zone, then fetch epw
for i in range(50):
    for cz in df["CZ"].unique():
        # Get the cities in each climate zone
        czdf = df[df["CZ"] == cz]

        # make sure the nth most populous city is defined for this cz
        if i < len(czdf):
            # Get the lat lon and try to download epw
            city = czdf.iloc[i]
            lat = city["lat"]
            lng = city["lng"]
            try:
                time.sleep(random.random()*5+1)
                city_epw = Epw.from_nrel(lat=lat,lon=lng)
            except BaseException as e:
                print(e)
                print("FAILED:",city["city"])
                continue
            else:
                # Save the epw and add it to the array
                print("SUCCESS",city["city"])
                slug = f"cityidx_{len(top_cities):04d}_{city_epw.name}"
                out_path = Path("./global_epws_indexed") / slug
                city_epw.save(out_path)
                city["slug"] = slug
                city["idx"] = len(top_cities)
                top_cities.loc[len(top_cities)] = city

# save the results and paths
top_cities.to_csv(Path("./global_epws_indexed.csv"))

INFO:pyumi.epw:Getting weather file: RUS_Omsk.286980_IWEC


SUCCESS Surgut


INFO:pyumi.epw:Getting weather file: CHN_Heilongjiang.Hailun.507560_CSWD


SUCCESS Suihua


INFO:pyumi.epw:Getting weather file: CHN_Tibet.Pagri.557730_SWERA


...

SUCCESS Gwalior


INFO:pyumi.epw:Getting weather file: GHA_Accra.654720_SWERA


SUCCESS Jos


INFO:pyumi.epw:Getting weather file: GHA_Accra.654720_SWERA


SUCCESS Aba


### Old Loop for ResStock 'cities.csv' 

In [ ]:
%%capture
cities = pd.read_csv("./cities.csv")
city_map = {}
for ix, row in cities.iterrows():
    print(f"Fetching Epw for {row['city']}")
    lat, lon = row["latitude"], row["longitude"]
    city_epw = Epw.from_nrel(lat, lon)
    """In case we want to mutate the epw to create variations per city"""
    dbt = np.array(city_epw.dry_bulb_temperature.values)
    dpt = np.array(city_epw.dew_point_temperature.values)
    rh = np.array(city_epw.relative_humidity.values)
    wind_dir = np.array(city_epw.wind_direction.values)
    wind_speed = np.array(city_epw.wind_speed.values)
    rad_norm= np.array(city_epw.direct_normal_radiation.values)
    rad_hor = np.array(city_epw.diffuse_horizontal_radiation.values)

    """Save it"""
    slug = f"cityidx_{ix:04d}_{city_epw.name}"
    out_path = Path("./city_epws_indexed") / slug
    city_epw.save(out_path)
    city_map[row['city']] = {
        'epw': slug,
        'idx': ix
    }
    print(f"Saved EPW for {row['city']}: {out_path}")

with open('city_map.json', 'w') as f:
    json.dump(city_map, f)

INFO:pyumi.epw:Getting weather file: CZ09RV2
INFO:pyumi.epw:Getting weather file: USA_NY_New.York-Central.Park.725033_TMY3
INFO:pyumi.epw:Getting weather file: USA_FL_Southwest.Florida.Intl.AP.722108_TMY3
INFO:pyumi.epw:Getting weather file: USA_CA_Stockton.Metro.AP.724920_TMY3
INFO:pyumi.epw:Getting weather file: USA_IL_Cahokia.AP.725314_TMY3
INFO:pyumi.epw:Getting weather file: USA_TX_Houston-William.P.Hobby.AP.722435_TMY3
INFO:pyumi.epw:Getting weather file: USA_CA_Santa.Ana-John.Wayne.AP.722977_TMY3
INFO:pyumi.epw:Getting weather file: USA_OK_Tulsa.723560_TMY2
INFO:pyumi.epw:Getting weather file: USA_OH_Dayton-Wright.Patterson.AFB.745700_TMY
INFO:pyumi.epw:Getting weather file: USA_MI_Grand.Rapids-Kent.County.Intl.AP.726350_TMY
INFO:pyumi.epw:Getting weather file: USA_GA_Dekalb.Peachtree.AP.722196_TMY3
INFO:pyumi.epw:Getting weather file: USA_WA_Snohomish.County.AP.727937_TMY3
INFO:pyumi.epw:Getting weather file: USA_MD_Baltimore-Washington.Intl.AP.724060_TMY
INFO:pyumi.epw:Getting